In [1]:
import base64
import requests
import os
import pandas as pd

In [ ]:
# Replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key
api_key = "YOUR_OPENAI_API_KEY"

# Function to encode the image to base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Replace 'path_to_your_image.jpg' with the path to your actual image file
# images path
image_file_path = os.fsencode(r"C:/Users/Jo/Documents/nitin_chatgpt_project/melanoma_project/melanoma_250/")
# image_path = r""

In [3]:
# empty list of dictionaries
data = []

# run for first x files in the directory
for file in os.listdir(image_file_path):
    # print(os.fsdecode(file))
    # filename = os.fsdecode(file)
    dict = {}
    # TODO: check if file is in the list of files in bcc_scc_dataframe.csv
    if ('.jpg' in file.decode('ascii')):
        base64_image = encode_image(os.path.join(image_file_path, file))
        # base64_image = encode_image(image_path)
        headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        }

        payload = {
          "model": "gpt-4o",
          "messages": [
            {
              "role": "user",
              "content": [
                {"type": "text", "text": '''This is an image from a patient. 
                
                    Based on the image, does the patient have A) Melanoma, B) No Melanoma? 
                    
                    Only output either A or B'''},
                
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                  }
                }
              ]
            }
          ],
          "max_tokens": 300
        }

        # Make the API request and print out the response
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        # parse out chatgpt message that we care about: add filename and the corresponding chatgpt response to dataframe
        try:
            print(file.decode('ascii'), ": ", response.json().get('choices')[0].get('message').get('content'))
            dict['filename'] = file.decode('ascii')
            dict['result'] = response.json().get('choices')[0].get('message').get('content')
            data.append(dict)
        # got an empty response rarely
        except TypeError:
            print(file.decode('ascii'), ": ", "")
            dict['filename'] = file.decode('ascii')
            dict['result'] = ""
            data.append(dict)
        # for anything else that I didn't expect
        except:
            print("Unresolved error: excluding the data")

0_m_or_nm.jpg :  A
100_m_or_nm (98).jpg :  A
101_m_or_nm (42).jpg :  A
102_m_or_nm (23).jpg :  A
103_m_or_nm (11).jpg :  A
104_m_or_nm (19).jpg :  A
105_m_or_nm (56).jpg :  A
106_m_or_nm (8).jpg :  A) Melanoma
107_m_or_nm (13).jpg :  A
108_m_or_nm (1).jpg :  A
109_m_or_nm (43).jpg :  A
10_m_or_nm.jpg :  A
110_m_or_nm (26).jpg :  A
111_m_or_nm (107).jpg :  A
112_m_or_nm (58).jpg :  I'm sorry, but I cannot make a medical diagnosis based on this image. It is important to consult a healthcare professional for an accurate evaluation.
113_m_or_nm (13).jpg :  A
114_m_or_nm (118).jpg :  A
115_m_or_nm (44).jpg :  A
116_m_or_nm (30).jpg :  A
117_m_or_nm (56).jpg :  A
118_m_or_nm (6).jpg :  A
119_m_or_nm (5).jpg :  A
11_m_or_nm.jpg :  B) No Melanoma
120_m_or_nm (74).jpg :  B
121_m_or_nm (70).jpg :  A
122_m_or_nm (118).jpg :  A
123_m_or_nm (46).jpg :  A
124_m_or_nm (21).jpg :  A
125_m_or_nm (96).jpg :  A
126_m_or_nm (91).jpg :  A
127_m_or_nm (97).jpg :  A
128_m_or_nm (82).jpg :  A
129_m_or_nm (12)

In [4]:
# write all responses to a file
final_data = pd.DataFrame(data)
final_data.to_csv("C:/Users/Jo/Documents/nitin_chatgpt_project/melanoma_project/melanoma_data/chatgpt_250.csv", sep=',', index=False, encoding='utf-8')